In [18]:
import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [19]:
Root = os.getcwd()
os.makedirs('Images_Folder', exist_ok=True)
os.makedirs('Result_Folder', exist_ok=True)

In [20]:
def Convert_to_PNG():
    os.chdir(os.path.join(Root,'Images_Folder'))
    image_patterns = ["*.jpeg", "*.jpg"]
    for pattern in image_patterns:
        for image_file in glob.glob(pattern):
            if not image_file.endswith(".png"):
                new_file = os.path.splitext(image_file)[0] + ".png"
                os.rename(image_file, new_file)
    os.chdir(Root)

In [21]:
def Image_Show(Image):
    plt.imshow(Image, cmap='gray')
    plt.axis('off')
    plt.show() 

In [22]:
def Convert_Image_To_Matrix(Image_Name):
    if Image_Name in os.listdir('Images_Folder'):
        Image = cv2.imread(os.path.join('Images_Folder', Image_Name))
        Gray_Image = np.zeros((Image.shape[0], Image.shape[1])) # Tạo ma trận 0 bằng kích thước với ma trận xuất ra từ hình ảnh
        for Row in range(Image.shape[0]): # chạy qua các hàng
            for Col in range(Image.shape[1]): # chạy qua các cột
                Pixel = Image[Row, Col]  # Lấy các giá trị Pixel của định dạng BRG
                Gray_Pixel = Pixel[0] * 0.114 + Pixel[1] * 0.587 + Pixel[2] * 0.299 # Chuyển sang Pixel của ảnh định dạng Gray
                Normalization = Gray_Pixel / 255 # Chuẩn hóa dữ liệu của mỗi Pixel bằng cách chia cho 255
                Gray_Image[Row, Col] = Normalization # Thêm các giá trị vào ảnh Gray
        return Gray_Image
    else: print("Picture not found!!!!")